# 1. Dataset y Preprocesamiento

**- ¿Por qué es necesario redimensionar las imágenes a un tamaño fijo para una MLP?**



Al crear un modelo MPL con capas lineales, antes de entrenar, definimos un número fijo de entradas ya que aprenden pesos fijos para cada posición del vector que se irán actualizando, para cada neurona. Si el input son imágenes de distintas dimensiones, después del `flatten` tendríamos vectores de diferente tamaño y obtendríamos un error de dimension porque no tendríamos coincidencia de los pesos de la primera capa que siempre espera un tamaño fijo.


**- ¿Qué ventajas ofrece Albumentations frente a otras librerías de transformación como `torchvision.transforms`?**

Albumentations es una librería que nos permite hacer transformaciónes de imágenes como técnica de data agumentation y cuenta con una documentación bien detallada con varios ejemplos para aprender a usar sus funciones.\
 Contiene funciones básicas como rotación, traslación y normalización pero además posee otras funcionalidades más complejas que permiten distorsionar la información para generar nuevas entradas como . Además, esta librería no solo funciona con simples imágenes en 2D si no que también permite trabajar con imágenes en 3D (como el caso de las imágenes médicas) y con videos. 

**- ¿Qué hace `A.Normalize()`? ¿Por qué es importante antes de entrenar una red?**

`A.Normalize()` es una transformación de Albumentations que aplica varias técnicas de normalización a una imagen para normalizar los valores de sus pixeles. La técnica de normalización específica (standard, image, image_per_channel, min_max, min_max_per_channel) puede seleccionarse con el parámetro normalization. En el caso de la standard se aplica: `img = (img - mean * max_pixel_value) / (std * max_pixel_value)` donde esos parámetros se deben asignar. \
Esto es importante antes de entrenar una red ya que acelera el entrenamiento y evita que entradas de valores muy extremos tengan mucho peso al generar el gradiente. MEJORAR ESTA PARTE


**- ¿Por qué convertimos las imágenes a `ToTensorV2()` al final de la pipeline?**

Este es el último paso de la data aumentation y lo debemos hacer porque Albumentations trabaja con imágenes como numpy arrays pero nosotros para entrenar la red en PyTorch necesitamos de tensores, entonces con esa línea de código logramos convertir el array de la imágen después de sus respectivas transformaciones en el formato correcto para poder entrenar la red.

# 2. Arquitectura del Modelo

**- ¿Por qué usamos una red MLP en lugar de una CNN aquí? ¿Qué limitaciones tiene?**

NO ENTIENDO LA PREGUNTA, TODAVIA NO VIMOS CNN.\
Para la clasificación y procesamiento de imágenes una CNN suele ser la solución más elegída. En este caso usamos un MLP que consiste en un modelo más simple y fácil de implementar, dado que las imágenes se redimensionan a un tamaño pequeño (64x64x3) para poder manejar la cantidad de datos, pero por lo general no es una estrategia elegida dado que una imagen tiene patrones espaciales que la caracterizan (como su textura o bordes específicos) que en este caso, al convertirlo en un vector 1D después del ´flatten´ se pierde esa información estructural que puede ser importante en el caso de querer detectar por ejemplo anomalías en la piel o en muchos casos médicos.

**- ¿Qué hace la capa `Flatten()` al principio de la red?**

La capa ´flatten()´ se encarga de convertir la imágen que está en 2D y varios canales (por lo general 3: R,G,B) en un vector unidimensional ya que al usar un MLP, las capas lineares solo admiten vectores de entrada. En nuestro caso, al trabajar con imágenes de 64x64 en los tres canales, la imagen después del flatten resulta en un vector de 64x64x3 elementos, es decir 12.288 elementos.

**- ¿Qué función de activación se usó? ¿Por qué no usamos `Sigmoid` o `Tanh`?**

En el modelo simple se implementó la función de activación ReLU, con la forma convencional de una rampa que toma el valor 0 si la entrada x es negativa y lo mantiene igual si es positiva. Esta es la función estándar para redes neuronales porque, además de ser simple de implementar, evita el problema del "vanishing gradient".\
Este problema ocurre cuando los gradientes se vuelven muy pequeños impidiendo que los pesos se actualicen correctamente, es decir, la red deja de aprender. Al usar funciones como Sigmoid y Tanh, cuyas derivadas son menores a 1, el vanishing gradient es un problema. Al propagarse hacia atrás a través de muchas capas, esas derivadas se multiplican y se achican más en cada paso, haciendo que los gradientes se desvanezcan.

**- ¿Qué parámetro del modelo deberíamos cambiar si aumentamos el tamaño de entrada de la imagen?**

Si aumentamos el tamaño de la imagen de entrada, deberíamos modificar el `input_size=64*64*3` que representa la nueva cantidad de elementos que componenen al vector de 1D después de la capa de flatten para poder trabajar con la capa linear del MLP. Otra opción es, en el caso de tener imágenes de mayor tamaño y querer trabajar con menos elementos, podemos redimensionar la imágen.

# 3. Entrenamiento y Optimización

**- ¿Qué hace `optimizer.zero_grad()`?**

La función `optimizer.zero_grad()` en PyTorch se usa para reiniciar (poner en cero) los gradientes acumulados de todos los parámetros del modelo antes de calcular los nuevos gradientes en el paso de backpropagation. Esto es necesario para evitar acumular el gradiente de los cálculos anteriores en el nuevo paso.

**- ¿Por qué usamos `CrossEntropyLoss()` en este caso?**

**- ¿Cómo afecta la elección del tamaño de batch (`batch_size`) al entrenamiento?**

El batch size (BS) 

**- ¿Qué pasaría si no usamos `model.eval()` durante la validación?**

`model.eval()` se utiliza durante etapa de inferencia y validación. Funciona deshabilitando comportamientos específicos del entrenamiento y utilizando los parámetros aprendidos para obtener resultados consistentes. Por ejemplo, si la red incluye dropout, en entrenamiento es una herramienta muy útil contra el overfitting, ya que al apagar neuronas aleatoriamente hace que neuronas con pesos pequeños ganen protagonismo. Por otro lado, durante la validación, no apaga neuronas ya que se busca evaluar el comportamiento de toda la red.

# 4. Validación y Evaluación

**- ¿Qué significa una accuracy del 70% en validación pero 90% en entrenamiento?**

El *accuracy* es una métrica que evalúa el porcentaje de predicciones correctas (TP+TN/all data). Esta métrica es fácil de interpretar pero no es siempre la más útil, específicamente en el caso de clasificaciones binareas desbalanceadas.\
Tener un rendimiento tan alto en entrenamiento y sustancialmente más bajo en validación (un 20% menos) sugiere que estamos en presencia de un caso de *overfitting*. En este caso el modelo se ajusta demasiado bien a la distribución de datos de entrenamiento (aprende el patrón o detalles específicos) y deja de generalizar bien, y es por eso que al presentarle datos nuevos el desempeño es tanto peor.

**- ¿Qué otras métricas podrían ser más relevantes que accuracy en un problema real?**

Las métricas más relevantes depende del problema a resolver, suele resultar útil complementar la métrica de accuracy con otras como:
- Precision: proporción de predicciones positivas que fueron correctas. Un modelo con alta precisión minimiza falsos positivos.
- Recall: Los positivos reales que detectó el modelo. Importa dependiendo del costo de los falsos negativos.
- F1 score: Balance entre precisión y recall, relevante cuando el costo de los errores es similar en ambas direcciones.



**- ¿Qué información útil nos da una matriz de confusión que no nos da la accuracy?**

**- En el reporte de clasificación, ¿qué representan `precision`, `recall` y `f1-score`?**

# 5. TensorBoard y Logging


**- ¿Qué ventajas tiene usar TensorBoard durante el entrenamiento?**